The goal of this notebook is the replicate an existing Curve pool (stableswap).

In [1]:
import sys, os
sys.path.append("../../")

from utils.contract_utils import init_contract
from utils.network_utils import configure_network

import brownie
from brownie.network import chain

configure mainnet to alchemy

In [2]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']

In [3]:
configure_network(
    node_provider_https=f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}",
    network_name='mainnet'
)

In [4]:
brownie.network.connect('mainnet-fork')

Attached to local RPC client listening at '127.0.0.1:8545'...


/Users/swadhinnanda/Projects/git/on-chain-analytics/venv/lib/python3.7/site-packages/brownie/network/main.py:46: BrownieEnvironmentWarning: Development network has a block height of 13244581
  BrownieEnvironmentWarning,


Load pool contract:

In [5]:
pool_contract_addr = "0x9D0464996170c6B9e75eED71c68B99dDEDf279e8"  # cvxcrv pool addr

pool_contract = init_contract(contract_addr=pool_contract_addr)
pool_contract.info()

  @title StableSwap
  @notice 2 coin pool implementation with no lending
  @author Curve.Fi
  @details Optimized to only support ERC20's with 18
           decimals that return True/revert



In [6]:
initial_A = pool_contract.initial_A()
initial_A_time = pool_contract.initial_A_time()
A = pool_contract.A()
A_precise = pool_contract.A_precise()
A_precision = int(A_precise/A)

print(f"pool param details:")
print(f"initial_A: {initial_A}")
print(f"initial_A_time: {initial_A_time}")
print(f"A: {A}")
print(f"A_precise: {A_precise}")
print(f"A_precision: {A_precision}")

pool param details:
initial_A: 5000
initial_A_time: 0
A: 50
A_precise: 5000
A_precision: 100


## Ramp pool to a different A.

Ramp parameters:

In [11]:
future_A = int(100)
t0 = chain.time()
t1 = int(t0 + 7 * 86400)

Get Admin address:


In [12]:
sender = brownie.accounts.at(init_contract("0xb9fc157394af804a3578134a6585c0dc9cc990d4").admin(), force=True)
sender

<Account '0x7EeAC6CDdbd1D0B8aF061742D41877D7F707289a'>

Ramp pool:

In [13]:
print(f"Ramp parameters: _future_A: {future_A}, _future_time: {t1}, msg.sender: {sender}, pool_contract: {pool_contract_addr}")
pool_contract.ramp_A(future_A, t1, {"from": sender})

Ramp parameters: _future_A: 100, _future_time: 1632661895, msg.sender: 0x7EeAC6CDdbd1D0B8aF061742D41877D7F707289a, pool_contract: 0x9D0464996170c6B9e75eED71c68B99dDEDf279e8
Transaction sent: 0x426d43e10b438a296f2681d77ff1c5da8043496cd0259f673ada39b86c664472
  Gas price: 0.0 gwei   Gas limit: 12000000   Nonce: 898
  Vyper_contract.ramp_A confirmed   Block: 13244583   Gas used: 77174 (0.64%)



<Transaction '0x426d43e10b438a296f2681d77ff1c5da8043496cd0259f673ada39b86c664472'>

Sleep for some time and then mine the chain to get new state.

In [14]:
chain.sleep(t1)
chain.mine()

13244584

Check if ramp was successful:

In [15]:
initial_A = pool_contract.initial_A()
initial_A_time = pool_contract.initial_A_time()
A = pool_contract.A()
A_precise = pool_contract.A_precise()
A_precision = int(A_precise/A)

print(f"pool param details:")
print(f"initial_A: {initial_A}")
print(f"initial_A_time: {initial_A_time}")
print(f"A: {A}")
print(f"A_precise: {A_precise}")
print(f"A_precision: {A_precision}")

pool param details:
initial_A: 5000
initial_A_time: 1632057097
A: 100
A_precise: 10000
A_precision: 100


Disconnect:

In [16]:
brownie.network.disconnect()